In [2]:
class Link:
    """A linked list.

    >>> s = Link(1)
    >>> s.first
    1
    >>> s.rest is Link.empty
    True
    >>> s = Link(2, Link(3, Link(4)))
    >>> s.first = 5
    >>> s.rest.first = 6
    >>> s.rest.rest = Link.empty
    >>> s                                    # Displays the contents of repr(s)
    Link(5, Link(6))
    >>> s.rest = Link(7, Link(Link(8, Link(9))))
    >>> s
    Link(5, Link(7, Link(Link(8, Link(9)))))
    >>> print(s)                             # Prints str(s)
    <5 7 <8 9>>
    """
    empty = ()

    def __init__(self, first, rest=empty):
        assert rest is Link.empty or isinstance(rest, Link)
        self.first = first
        self.rest = rest

    def __repr__(self):
        if self.rest is not Link.empty:
            rest_repr = ', ' + repr(self.rest)
        else:
            rest_repr = ''
        return 'Link(' + repr(self.first) + rest_repr + ')'

    def __str__(self):
        string = '<'
        while self.rest is not Link.empty:
            string += str(self.first) + ' '
            self = self.rest
        return string + str(self.first) + '>'

class Tree:
    """
    >>> t = Tree(3, [Tree(2, [Tree(5)]), Tree(4)])
    >>> t.label
    3
    >>> t.branches[0].label
    2
    >>> t.branches[1].is_leaf()
    True
    """
    def __init__(self, label, branches=[]):
        for b in branches:
            assert isinstance(b, Tree)
        self.label = label
        self.branches = list(branches)

    def is_leaf(self):
        return not self.branches

    def map(self, fn):
        """
        Apply a function `fn` to each node in the tree and mutate the tree.

        >>> t1 = Tree(1)
        >>> t1.map(lambda x: x + 2)
        >>> t1.map(lambda x : x * 4)
        >>> t1.label
        12
        >>> t2 = Tree(3, [Tree(2, [Tree(5)]), Tree(4)])
        >>> t2.map(lambda x: x * x)
        >>> t2
        Tree(9, [Tree(4, [Tree(25)]), Tree(16)])
        """
        self.label = fn(self.label)
        for b in self.branches:
            b.map(fn)

    def __contains__(self, e):
        """
        Determine whether an element exists in the tree.

        >>> t1 = Tree(1)
        >>> 1 in t1
        True
        >>> 8 in t1
        False
        >>> t2 = Tree(3, [Tree(2, [Tree(5)]), Tree(4)])
        >>> 6 in t2
        False
        >>> 5 in t2
        True
        """
        if self.label == e:
            return True
        for b in self.branches:
            if e in b:
                return True
        return False

    def __repr__(self):
        if self.branches:
            branch_str = ', ' + repr(self.branches)
        else:
            branch_str = ''
        return 'Tree({0}{1})'.format(self.label, branch_str)

    def __str__(self):
        def print_tree(t, indent=0):
            tree_str = '  ' * indent + str(t.label) + "\n"
            for b in t.branches:
                tree_str += print_tree(b, indent + 1)
            return tree_str
        return print_tree(self).rstrip()

In [ ]:
def sum_nums(lnk):
  if lnk == Link.empty:
    return 0
  return lnk.first + sum_nums(lnk.rest)

a = Link(1, Link(6, Link(7)))
sum_nums(a)

14

In [ ]:
def multiply_lnks(lst_of_lnks):
  product = 1
  for lnk in lst_of_lnks:
    if lnk is Link.empty:
      return Link.empty
    product += lnk.first
  lst_of_lnks_rests = [lnk.rest for lnk in lst_of_lnks]
  return Link(product, multiply_lnks(lst_of_lnks_rests))

In [ ]:
def filter_link(link, f):
  while link is not Link.empty:
    if f(link.first):
      yield link.first
    link = link.rest

link = Link(1, Link(2, Link(3)))
g = filter_link(link, lambda x: x % 2 == 0)
next(g)

2

In [ ]:
def filter_no_iter(link, f):
  if link is Link.empty:
    return
  elif f(link.first):
    yield link.first
  yield from filter_no_iter(link.rest, f)

link = Link(1, Link(2, Link(3)))
list(filter_no_iter(link, lambda x: x % 2 != 0))

[1, 3]

In [ ]:
def make_even(t):
  if t.label % 2 != 0:
      t.label = t.label + 1
  for b in t.branches:
    make_even(b)
    

t = Tree(1, [Tree(2, [Tree(3)]), Tree(4), Tree(5)])
make_even(t)
t.label

2

In [ ]:
def square_tree(t):
  t.label = t.label ** 2
  for b in t.branches:
    square_tree(b)

t = Tree(1, [Tree(2, [Tree(3)]), Tree(4), Tree(5)])
square_tree(t)
t.branches[0].branches[0].label


9

In [ ]:
def find_path(t, entry):
  if t.label == entry:
    return [t.label]
  for b in t.branches:
    path = find_path(b, entry)
    if path:
      return [t.label] + path

tree_ex = Tree(2, [Tree(7, [Tree(3), Tree(6, [Tree(5), Tree(11)])]), Tree(1)])
find_path(tree_ex, 5)

[2, 7, 6, 5]

In [ ]:
def average(t):
  def sum_helper(t):
    total, count = 0, 0
    for b in t.branches:
      total += b.label
      count += 1
    return total, count
  total, count = sum_helper(t)
  return total / count

t0 = Tree(0, [Tree(1), Tree(2, [Tree(3)])])
average(t0)

t1 = Tree(8, [t0, Tree(4)])
average(t1)

2.0

In [6]:
def combine_tree(t1, t2, combiner):
  combined = [combine_tree(b1, b2, combiner) for b1, b2 in zip(t1.branches, t2.branches)]
  return Tree(combiner(t1.label, t2.label), combined)
  

a = Tree(1, [Tree(2, [Tree(3)])])
b = Tree(4, [Tree(5, [Tree(6)])])
combined = combine_tree(a, b, mul)
combined.label

NameError: ignored

In [7]:
def alt_tree_map(t, map_fn):
  def helper(t, depth):
    if depth % 2 == 0:
      label = map_fn(t.label)
    else:
      label = t.label
    branches = [helper(b, depth + 1) for b in t.branches]
    return Tree(label, branches)
  return helper(t, 0)

t = Tree(1, [Tree(2, [Tree(3)]), Tree(4)])
negate = lambda x: -x
alt_tree_map(t, negate)

Tree(-1, [Tree(2, [Tree(-3)]), Tree(4)])